In [ ]:
from huggingface_hub import InferenceClient
import base64
import re
import pandas as pd
from sklearn.metrics import mean_absolute_error
import time
import os
import torch
import openai
device = "cuda" if torch.cuda.is_available() else "cpu"
# clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
client = InferenceClient(api_key="")
api_key = ""

# Path
csv_path = "/Users/zhenglinning/Desktop/Brown/CSCI2470/DL_Final_Project/data/nutrition5k_reconstructed/metadata/test_ids.csv"
labels_path = "/Users/zhenglinning/Desktop/Brown/CSCI2470/DL_Final_Project/data/nutrition5k_reconstructed/labels/labels.csv"
gpt_response_path = "gpt_response.csv"
gpt_predict_path = "gpt_predict.csv"

df = pd.read_csv(csv_path, header=None)
image_ids = df[0].tolist() # list of image ids in test_ids.csv
# true labels
label_df = pd.read_csv(labels_path, header=None)
filtered_label_df = label_df[label_df[0].isin(image_ids)]
id_dict = filtered_label_df.set_index(0).T.to_dict('list')
id_dict = {key: [float(value) for value in values] for key, values in id_dict.items()}

# save to csv to prevent crashing
if os.path.exists(gpt_response_path):
    gpt_response = pd.read_csv(gpt_response_path, index_col=0).to_dict(orient="list")
    gpt_response = {key: [float(x) for x in value] for key, value in gpt_response.items()}
else:
    gpt_response = {}

# remove processed ids
remaining_ids = [i for i in image_ids if i not in gpt_response]
count = 0

In [56]:
def parse_res(response):
    try:
        final_response = re.search(r'\{.*\}', response).group() # extract the response
        if not final_response:
            final_response = "Error"
    except Exception as e:
        final_response = "Error"
    return final_response

In [59]:
for i in remaining_ids[:100]:
    if count == 10:
        pd.DataFrame.from_dict(gpt_response, orient="index").to_csv(gpt_response_path)
        time.sleep(100)
        count = 0
    
    image_path = "../data/nutrition5k_reconstructed/images/" + i + ".jpeg"
    with open(image_path, "rb") as f:
        base64_image = base64.b64encode(f.read()).decode("utf-8")

    image_url = f"data:image/png;base64,{base64_image}"

    messages = [
        {
            "role": "system",
            "content": """You are a nutritionist. You will be given an image of food. Analyze the food in the image and tell me what ingredients are used. """
        },
        {
            "role": "user",
            "content": """I will provide an image of food and you will analyze the food in the image and provide its ingredients. Your response must strictly follow this format: {ingredients: [list of ingredients]}. Please provide the image of the food you would like me to analyze."""
        },
        {
            "role": "assistant",
            "content": """I will answer your questions with the following format: {ingredients: [list of ingredients]}. Please provide the image of the food you would like me to analyze."""
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Here is an image of food I would like you to analyze. Remember to follow the format."
                },
                {
                    "type": "image_url",
                    "image_url": {"url": image_url},
                }
            ]
        }
    ]


    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct", 
        messages=messages, 
        max_tokens=500
    )

    response = completion.choices[0].message["content"] # get response
    response = parse_res(response)
    gpt_response[i] = response
    print(i, response)
    count += 1

dish_1558722171 {ingredients: Cauliflower}
dish_1558031237 {ingredients: melon}
dish_1563565354 {ingredients: fish with petals of creamy sauce and steam}
dish_1563898084 {ingredients: omega-3 fatty acid, dietary potassium, protein, vitamins b, magnesium, vitamins, fiber, dietary fiber, vitamin b6, zinc, calcium, magnesium, vitamin C content, vitamin A, manganese, proteases, bromelain, thromboplastinincentration, aspirin-like activity, vitamin B6, choline, fiber-rich cactus, cortisol, bromelain, low-fat cactus, whey protein, whey protein isolate, condensed water, hydrolyzed adipose tissue, dry milk solids, salt, osmotic sugar, high cellulose content, golgi apparatus, lipolysis, cornstarch, pig fatin, allergy friendly, antioxidants, sweet cherry dark red cherry, essential ammonia, mRNA expression, ultra-sonic homogenization, highly efficient microbes, glucase activity, leukemia, pancreatic insufficiency, glycogen, magenta bond, pectin sintethisol, tsJuliezkonzenzo rubrontene, Fedeiveluch

In [51]:
def parse_res2(response):
    try:
        final_response = re.search(r'\{.*\}', response).group() # extract the response
        if final_response:
            values = re.findall(r":\s*([\d.]+)", final_response) # extract the values
            final_response = [float(i) for i in values] # convert to float
        else:
            final_response = [0, 0, 0, 0, 0]
    except Exception as e:
        final_response = [0, 0, 0, 0, 0]
    return final_response

In [ ]:
dish_ingredients = {}
with open('gpt_response.csv', 'r') as f:
    for line in f:
        if not line:
            continue
        try:
            dish_id, ingredients = line.split(',', 1)
            ingredients = ingredients.strip()
            dish_id = dish_id.strip()
            dish_ingredients[dish_id] = ingredients
        except Exception as e:
            print(e)
dish_ingredients.__delitem__("")

{'dish_1558722171': '{ingredients: Cauliflower}', 'dish_1558031237': '{ingredients: melon}', 'dish_1563565354': '{ingredients: fish with petals of creamy sauce and steam}', 'dish_1563898084': '"{ingredients: omega-3 fatty acid, dietary potassium, protein, vitamins b, magnesium, vitamins, fiber, dietary fiber, vitamin b6, zinc, calcium, magnesium, vitamin C content, vitamin A, manganese, proteases, bromelain, thromboplastinincentration, aspirin-like activity, vitamin B6, choline, fiber-rich cactus, cortisol, bromelain, low-fat cactus, whey protein, whey protein isolate, condensed water, hydrolyzed adipose tissue, dry milk solids, salt, osmotic sugar, high cellulose content, golgi apparatus, lipolysis, cornstarch, pig fatin, allergy friendly, antioxidants, sweet cherry dark red cherry, essential ammonia, mRNA expression, ultra-sonic homogenization, highly efficient microbes, glucase activity, leukemia, pancreatic insufficiency, glycogen, magenta bond, pectin sintethisol, tsJuliezkonzenzo

In [ ]:
count = 0
all_the_keys = [k for k in dish_ingredients.keys()]
# find the keys that have valid responses
valid_keys = [k for k in all_the_keys if dish_ingredients[k] != "Error"]
# get target values
y_true = dict((k, id_dict[k]) for k in valid_keys)
llama_response = dict((k, dish_ingredients[k]) for k in valid_keys)
gpt_predict_path = "gpt_predict.csv"

86


In [100]:
y_pred = {}
for i in valid_keys:
    if count == 10:
        pd.DataFrame.from_dict(y_pred, orient="index").to_csv(gpt_predict_path)
        time.sleep(100)
        count = 0
    food_description = llama_response[i]
    if food_description == "Error":
        response = ""
    else:
        messages = [
            {
                "role": "system",
                "content": (
                    "You are a nutritionist. Based on the food description, estimate its nutritional facts "
                    "(calories, mass, fat, carbs, protein) strictly in this format: "
                    "{calories: <calories>, mass: <mass>, fat: <fat>, carbs: <carbs>, protein: <protein>}. "
                    "Do not include additional text."
                )
            },
            {
                "role": "user",
                "content": f"The food description is: '{food_description}'. Provide nutritional facts in the specified format."
            }
        ]

        client = openai.OpenAI(
            api_key=api_key,  # This is the default and can be omitted
        )
        completion = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
        )
        response = completion.choices[0].message.content
        
    if response == "":
        y_pred[i] = [0, 0, 0, 0, 0]
    else:
        final_response = parse_res2(response)
        y_pred[i] = final_response
        if y_pred[i] == []:
            y_pred[i] = [0, 0, 0, 0, 0] # if no prediction from llama, placeholder value
        print(y_pred[i], i)
    count += 1

pd.DataFrame.from_dict(y_pred, orient="index").to_csv(gpt_predict_path) # save to csv again

[25.0, 100.0, 0.3, 5.0, 2.0] dish_1558722171
[60.0, 300.0, 0.3, 15.0, 1.5] dish_1558031237
[250.0, 150.0, 10.0, 5.0, 30.0] dish_1563565354
[350.0, 100.0, 10.0, 55.0, 12.0] dish_1563898084
[350.0, 300.0, 15.0, 15.0, 30.0] dish_1565986702
[250.0, 250.0, 8.0, 38.0, 4.0] dish_1566920365
[550.0, 250.0, 35.0, 30.0, 25.0] dish_1562009006
[300.0, 200.0, 15.0, 35.0, 4.0] dish_1559233715
[450.0, 350.0, 14.0, 65.0, 18.0] dish_1561578865
[400.0, 350.0, 12.0, 60.0, 15.0] dish_1562963289
[200.0, 250.0, 10.0, 10.0, 20.0] dish_1568305225
[150.0, 150.0, 0.5, 33.0, 3.0] dish_1561663541
[350.0, 200.0, 20.0, 25.0, 20.0] dish_1566851704
[450.0, 350.0, 15.0, 50.0, 30.0] dish_1563478751
[400.0, 250.0, 20.0, 40.0, 10.0] dish_1564675411
[250.0, 200.0, 10.0, 30.0, 15.0] dish_1566589933
[200.0, 250.0, 8.0, 28.0, 8.0] dish_1567785739
[230.0, 145.0, 4.0, 32.0, 15.0] dish_1563551134
[250.0, 100.0, 15.0, 5.0, 20.0] dish_1562703345
[42.0, 10.0, 3.3, 0.1, 3.0] dish_1563897938
[350.0, 250.0, 18.0, 10.0, 30.0] dish_1560

In [107]:
feature_names = ['calories', 'mass', 'fat', 'carbs', 'protein']

truth_values_per_feature = {feature: [] for feature in feature_names}
pred_values_per_feature = {feature: [] for feature in feature_names}

for dish_id in y_true:
    if dish_id in y_pred:
        true_values = y_true[dish_id]
        pred_values = y_pred[dish_id]
        for i, feature in enumerate(feature_names):
            truth_values_per_feature[feature].append(true_values[i])
            pred_values_per_feature[feature].append(pred_values[i])
    else:
        print(f"No prediction available for {dish_id}")

mae_per_feature = {}
for feature in feature_names:
    mae = mean_absolute_error(truth_values_per_feature[feature], pred_values_per_feature[feature])
    mae_per_feature[feature] = mae
    print(f"MAE for {feature}: {mae}")

overall_mae = sum(mae_per_feature.values()) / len(feature_names)
print(f"Overall MAE: {overall_mae}")



MAE for calories: 146.2213044651163
MAE for mass: 106.44186046511628
MAE for fat: 10.143527023255814
MAE for carbs: 13.440968895348837
MAE for protein: 10.72016538372093
Overall MAE: 57.39356524651164


In [105]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

all_true_values = []
all_pred_values = []

for i in range(len(feature_names)):
    all_true_values.append(truth_values_per_feature[feature_names[i]])
    all_pred_values.append(pred_values_per_feature[feature_names[i]])

all_true_values = np.array(all_true_values).T
all_pred_values = np.array(all_pred_values).T

scaler = MinMaxScaler()
scaler.fit(all_true_values)

true_values_normalized = scaler.transform(all_true_values)
pred_values_normalized = scaler.transform(all_pred_values)

true_values_flat = true_values_normalized.flatten()
pred_values_flat = pred_values_normalized.flatten()

overall_mae_normalized = mean_absolute_error(true_values_flat, pred_values_flat)
print(f"Overall MAE after normalization: {overall_mae_normalized}")


Overall MAE after normalization: 0.14349299921700584
